In [1]:
import sys
sys.path.append("ALL_IN_ONE/Pythia8")
sys.path.append("ALL_IN_ONE/LSD")
sys.path.append("ALL_IN_ONE/FIT_FUNC")

In [2]:
import os
from tqdm import tqdm
import ALL_IN_ONE.Pythia8.loop as loop
import ALL_IN_ONE.Pythia8.one_key_run as okr 
import ALL_IN_ONE.Pythia8.run_save as rs
import ALL_IN_ONE.Pythia8.analyse_data as ad
import ALL_IN_ONE.Pythia8.detection as detection
import ALL_IN_ONE.Pythia8.Mulity_process as mp
from datetime import datetime
import ALL_IN_ONE.Pythia8.combine as combine
# from detection import detect_percise_and_combine, detect_r_and_combine
# from analyse_data import see_ctau_2, print_max_min_log10_sin_theta_2

### This Func is made for FUTURE USE aimmed to speed up our simulations(Whose speed is Not Satisfying right now(2024.11.17)). 

### But it is not usable now

In [9]:
# from joblib import Parallel, delayed

# def worker(seed):
#     LLP_data_path = loop_mass_ctau_given_by_csv_multi(csv_file, br, seed, out_put_path, main41_path)
#     print('The Generation of LLPs is Completed')
#     completed_data_dir = detect_folder_files_no_calcu(LLP_data_path)[1]
#     print('The LLPs are Judged whether they are Detected or not')
#     final_files = combine_files_precise(completed_data_dir)
#     print('The Final Step is Over, See the .csv files for LLPs Completed Data')
#     return LLP_data_path, completed_data_dir, final_files

# def one_key_run_mass_ctau_given_by_csv_multi(csv_file, br, seed_array, out_put_path, main41_path):
#     results = Parallel(n_jobs=5)(delayed(worker)(seed) for seed in seed_array)
#     return results


## Attention:  

## The Units are CM and GEV

### The 'one_key_run_XXX' functions are all you need to use.



### Setting Parameters

In [10]:
c_mm = (3*10**5) # tau0's unit is mm/c, to convert the ultimate decay postion,
                            # we need to do sth to the ctau_typed_in 

br_lowerlim = -20 #ctau and br are in logspace
br_upperlim = -20
br_amount = 1
ctau_lowerlim = 1
ctau_upperlim = 30
ctau_amount = 50
mass = 1
seed_amount = 5
seed_array = (357993, 1234) #Pythia8 simulation seed

mass_lower_lim = 0.1 # Mass in in linspace
mass_upper_lim = 4
mass_amount = 20
br = 1

#### Same as before:  

##### The function 
```python
    one_key_run_mass_ctau_simple_ver() #Function to run an pythia8 program
```

takes 

```
br's lower / upper lim (10^x), br's amount, 

ctau's lower / upper lim (10^x), ctau's amount, 

mass, 

seed's amount, 

the Output folder's path, 

main41.cc's dir path
```
as input.

The Output is LOTS OF ```.csv``` Files with LLPs' Properties 

```
    Momentum in GeV, 
    
    lifetime(ctau or tau) in mm/C, 

    decay_postion in mm, mass in GeV 

    and so on
```

The path's format is listed below, you can change them directly to your real path to run the function

In [4]:
main131_path = '/home/srp/Documents/Pythia8/pythia8315/examples/'
# For Pythia8.31, main41 is now main131 but the path remains the same
out_path = '/home/srp/Documents/LLP_DATA/'
today = str(datetime.today().date())+'_Higgs_Scalar'
# today = '2025-06-07_2HDM-A'
print(today)


2025-08-20_Higgs_Scalar


#### New Function info:

The function
```python
    def one_key_run_mass_ctau_br_given_by_csv_main131(csv_file, br, seed_amount, out_path, main41_path)
```

takes a ```.csv file``` as input instead of the range of ```mass```, ```tau```, ```Br_XX```and etc. 

The function will read each line of ```.csv file``` to get `LLP's DATA Range`, ALONG WITH ```ALL THE Branching Ratio of OUR LLP``` and put this set of data into pythia8. 

Pythia8 will do the simulation, then output .csv file to the ``` out_path``` folder.

In [ ]:
# test_csv = '/media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Program/PRA/Github/position_read_analyse_1.3/ALL_IN_ONE/LSD/tau_theta_1e-05.csv'
# okr.one_key_run_by_csv_cross_section_main131_simple(test_csv, 1, 1, out_path, main131_path, today, 0)

Running Simulation...


  0%|          | 0/9 [00:01<?, ?it/s]


KeyboardInterrupt: 

In [9]:
csv_file = '/home/srp/Documents/program/stonerainphilo-Light_scalar_decay/Combined_Code/Higgs_Scalar_theta.csv'
mp.split_csv_into_different_parts(csv_file, 4)

CSV file has been split into 4 parts.


In [ ]:
test_for_mass_1 = okr.one_key_run_2HDM_cross_section_main131_lower_eff_all_detectors(csv_file, 1, 1, out_path, main131_path, today, 0)

In [5]:
print(today)

2025-08-20_Higgs_Scalar


In [12]:
from concurrent.futures import ProcessPoolExecutor
import os

# 假设 okr.one_key_run_by_csv_cross_section_main131 是你要并行运行的函数
from concurrent.futures import ProcessPoolExecutor

def run_task(args):
    _csv_file, _param1, _param2, _out_path, _main131_path, _date, _sleep_time = args
    return okr.one_key_run_2HDMA_cross_section_main131_lower_eff_all_detectors_B(_csv_file, _param1, _param2, _out_path, _main131_path, _date, _sleep_time)

if __name__ == "__main__":
    # 自定义 CPU 核心数量
    num_cores = 4  # 你可以将此值改为你想使用的核心数量

    # 定义任务参数
    # out_path = "output_path"  # 替换为实际的输出路径
    # main131_path = "main131_path"  # 替换为实际的路径

    tasks = [
        (f"{i+1}.csv", 1, 1, out_path, main131_path, today, 3)  # 生成任务参数
        for i in range(num_cores)  # 根据自定义核心数生成任务
    ]

    # 打印生成的任务
    for task in tasks:
        print(task)

    # 使用多核并行
    with ProcessPoolExecutor(max_workers=num_cores) as executor:
        results = list(executor.map(run_task, tasks))

    # 打印结果
    print("All tasks completed:", results)


run_task((csv_file, 1, 1, out_path, main131_path, today, 0))

('1.csv', 1, 1, '/home/srp/Documents/LLP_DATA/', '/home/srp/Documents/Pythia8/pythia8315/examples/', '2025-08-20_Higgs_Scalar', 3)
('2.csv', 1, 1, '/home/srp/Documents/LLP_DATA/', '/home/srp/Documents/Pythia8/pythia8315/examples/', '2025-08-20_Higgs_Scalar', 3)
('3.csv', 1, 1, '/home/srp/Documents/LLP_DATA/', '/home/srp/Documents/Pythia8/pythia8315/examples/', '2025-08-20_Higgs_Scalar', 3)
('4.csv', 1, 1, '/home/srp/Documents/LLP_DATA/', '/home/srp/Documents/Pythia8/pythia8315/examples/', '2025-08-20_Higgs_Scalar', 3)
Running Simulation...Running Simulation...Running Simulation...Running Simulation...





KeyError: 'mA'

In [8]:
from concurrent.futures import ProcessPoolExecutor
import os

# 假设 okr.one_key_run_by_csv_cross_section_main131 是你要并行运行的函数
from concurrent.futures import ProcessPoolExecutor

def run_task(args):
    _csv_file, _param1, _param2, _out_path, _main131_path, _date, _sleep_time = args
    return okr.one_key_run_by_csv_cross_section_main131_simple(_csv_file, _param1, _param2, _out_path, _main131_path, _date, _sleep_time)

if __name__ == "__main__":
    # 自定义 CPU 核心数量
    num_cores = 4  # 你可以将此值改为你想使用的核心数量

    # 定义任务参数
    # out_path = "output_path"  # 替换为实际的输出路径
    # main131_path = "main131_path"  # 替换为实际的路径

    tasks = [
        (f"{i+1}.csv", 1, 1, out_path, main131_path, today, 0)  # 生成任务参数
        for i in range(num_cores)  # 根据自定义核心数生成任务
    ]

    # 打印生成的任务
    for task in tasks:
        print(task)

    # 使用多核并行
    with ProcessPoolExecutor(max_workers=num_cores) as executor:
        results = list(executor.map(run_task, tasks))

    # 打印结果
    print("All tasks completed:", results)


run_task((csv_file, 1, 1, out_path, main131_path, today, 0))

('1.csv', 1, 1, '/home/srp/Documents/LLP_DATA/', '/home/srp/Documents/Pythia8/pythia8315/examples/', '2025-08-20_Higgs_Scalar', 0)
('2.csv', 1, 1, '/home/srp/Documents/LLP_DATA/', '/home/srp/Documents/Pythia8/pythia8315/examples/', '2025-08-20_Higgs_Scalar', 0)
('3.csv', 1, 1, '/home/srp/Documents/LLP_DATA/', '/home/srp/Documents/Pythia8/pythia8315/examples/', '2025-08-20_Higgs_Scalar', 0)
('4.csv', 1, 1, '/home/srp/Documents/LLP_DATA/', '/home/srp/Documents/Pythia8/pythia8315/examples/', '2025-08-20_Higgs_Scalar', 0)
Running Simulation...Running Simulation...Running Simulation...Running Simulation...





100%|██████████| 207/207 [6:23:33<00:00, 111.17s/it]  


The Generation of LLPs is Completed


  1%|▏         | 11/821 [00:13<16:34,  1.23s/it]/it]

Error with file: /home/srp/Documents/LLP_DATA/2025-08-20_Higgs_Scalar/LLP_data/mass_5.01e+00_ctau_2.20e+04_br_1.00e+00_seed_253910.csv
Error message: No columns to parse from file


...

100%|██████████| 829/829 [18:40<00:00,  1.35s/it]


The LLPs are Judged whether they are Detected or not, and calculated the cross section


100%|██████████| 782/782 [02:19<00:00,  5.59it/s]


The Final Step is Over, See the .csv files for LLPs Completed Data
All tasks completed: [('/home/srp/Documents/LLP_DATA/2025-08-20_Higgs_Scalar/LLP_data', '/home/srp/Documents/LLP_DATA/2025-08-20_Higgs_Scalar/Completed_llp_data_precise_cross_section', ('/home/srp/Documents/LLP_DATA/2025-08-20_Higgs_Scalar/2025-08-20_all_combined_precise_file.csv', '/home/srp/Documents/LLP_DATA/2025-08-20_Higgs_Scalar/2025-08-20_detected_combined_precise_file.csv')), ('/home/srp/Documents/LLP_DATA/2025-08-20_Higgs_Scalar/LLP_data', '/home/srp/Documents/LLP_DATA/2025-08-20_Higgs_Scalar/Completed_llp_data_precise_cross_section', ('/home/srp/Documents/LLP_DATA/2025-08-20_Higgs_Scalar/2025-08-20_all_combined_precise_file.csv', '/home/srp/Documents/LLP_DATA/2025-08-20_Higgs_Scalar/2025-08-20_detected_combined_precise_file.csv')), ('/home/srp/Documents/LLP_DATA/2025-08-20_Higgs_Scalar/LLP_data', '/home/srp/Documents/LLP_DATA/2025-08-20_Higgs_Scalar/Completed_llp_data_precise_cross_section', ('/home/srp/Docume

NameError: name 'csv_file' is not defined

#### To read the data

Now that you have done the simulation via pythia8, the output .csv files should be in your `output` folder. 

Below the folder, the file is sorted and filed by `date of the simulation`, so there will be a new folder like `2025-03-07`. In that `date folder`, there will be two folders, named`LLP_data`(store the orginal data), and `Completed_llp_data_precise` (store the detective data). 

AND THERE IS A `.csv` FILE, which is named like `2025-03-07_detected_combined_precise_file.csv`, it contains ONLY the LLPs can be DETECTED by CODEX-b.

In [ ]:
print(ad.print_max_min_log10_sin_theta_2('/Users/shiyuzhe/Documents/University/LLP/Second_Term/pythia8/BtoKa/Code/position_read_analyse_1.3/LLP_data_for_test/2025-2-16/2025-2-16_detected_combined_precise_file.csv'))

(-10.200000256677525, -12.400000474218762)


### The Detect Function

Functions::::
```python
def detect_r_and_combine(LLP_data_path) 

def detect_precise_and_combine(LLP_data_path)
```

Are functions to combine '''.csv files''' and judge whether LLP can be detected by CODEX-b or not

They will creat a ```.csv file``` at the dir of your LLP, which contains ALL DETECTED LLPS.

The function::::
```python 
def detect_r_and_combine(LLP_data_path)
```
 is merely a test function. You can play with it to get more data, but DO NOT use it for real analyzation

In [3]:
LLP_date_path = '/home/srp/Documents/LLP_DATA/2025-08-08_2HDM-A/LLP_data'
# detected_r = '/Users/shiyuzhe/Documents/University/LLP/Second_Term/pythia8/BtoKa/Code/position_read_analyse_1.3/LLP_data_for_test/2024-11-18_19/'
okr.calcu_cross_section_and_combine_files_CODEX_MATHUSLA_SHiP(LLP_date_path)

  1%|          | 15/1545 [46:47<77:56:22, 183.39s/it]

Error with file: /home/srp/Documents/LLP_DATA/2025-08-08_2HDM-A/LLP_data/mass_1.00e+00_ctau_9.07e-05_br_1.00e+00_seed_846798.csv
Error message: No columns to parse from file


 27%|██▋       | 424/1545 [22:12:41<58:28:53, 187.81s/it]

Error with file: /home/srp/Documents/LLP_DATA/2025-08-08_2HDM-A/LLP_data/mass_3.98e+00_ctau_2.28e+00_br_1.00e+00_seed_380686.csv
Error message: No columns to parse from file


 29%|██▉       | 453/1545 [23:40:27<57:17:09, 188.85s/it]

...

100%|██████████| 1545/1545 [80:37:22<00:00, 187.86s/it]   


The LLPs are Judged whether they are Detected or not, and calculated the cross section


100%|██████████| 1537/1537 [10:07<00:00,  2.53it/s]


The Final Step is Over, See the .csv files for LLPs Completed Data


('/home/srp/Documents/LLP_DATA/2025-08-08_2HDM-A/Completed_llp_data_precise_cross_section',
 ('/home/srp/Documents/LLP_DATA/2025-08-08_2HDM-A/2025-08-12_all_combined_precise_file.csv',
  '/home/srp/Documents/LLP_DATA/2025-08-08_2HDM-A/2025-08-12_detected_combined_precise_file.csv'))

In [ ]:
combinefile = '/media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Files/LLP_DATA/Original_DATA/'
combine.merge_csv_files(combinefile, combinefile + '/combined.csv')

All CSV files have been merged into /media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Files/LLP_DATA/Original_DATA/ALL_GOOD_DATA_2/combined.csv
